#License and Attribution

This notebook was developed by Emilio Serrano, Full Professor at the Department of Artificial Intelligence, Universidad Politécnica de Madrid (UPM), for educational purposes in UPM courses. Personal website: https://emilioserrano.faculty.bio/

📘 License: Creative Commons Attribution-NonCommercial-ShareAlike (CC BY-NC-SA)

You are free to: (1) Share — copy and redistribute the material in any medium or format; (2) Adapt — remix, transform, and build upon the material.

Under the following terms: (1) Attribution — You must give appropriate credit, provide a link to the license, and indicate if changes were made; (2) NonCommercial — You may not use the material for commercial purposes; (3) ShareAlike — If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.

🔗 License details: https://creativecommons.org/licenses/by-nc-sa/4.0/

# Prompt Engineering with LangChain: Building LLM-Powered Applications

In this notebook, we’ll explore how to build fast and modular Generative AI applications using LangChain and Groq.

[LangChain](https://www.langchain.com/) is a flexible framework designed to help developers integrate large language models (LLMs) into real-world applications. It provides tools for chaining together components like prompt templates, memory, agents, and access to external tools or data—making it ideal for building intelligent, composable workflows.

[Groq](https://groq.com/) offers an inference API optimized for ultra-low latency and high-throughput LLM execution. It enables real-time performance with state-of-the-art models such as LLaMA 3 and Gemma, making it an excellent option when responsiveness is critical.

⚠️ While we use Groq here for convenience and speed, LangChain is model-agnostic. You can easily swap Groq for other backends like OpenAI, Hugging Face, or Anthropic. You can also run open-source models locally on your own machine using tools like LM Studio, Ollama, or Text Generation WebUI—ideal for development without API limits or cloud dependency.

By combining LangChain’s modularity with a fast inference backend like Groq (or a local setup), you’ll learn how to prototype and deploy efficient, production-ready LLM-powered solutions.

Learning Objectives:

- Connect and use the Groq API via LangChain to run Large Language Models (LLMs).

- Start with a simple, direct prompt and see its limitations.

- Create structured prompts with roles (system, user) and templates to guide the model.

- Enforce JSON output for easy integration with other software.

- Understand and apply prompting techniques like Zero-shot, One-shot, and Few-shot.



# Environment Setup
First, we need to install the necessary libraries and configure our Groq API key.

No GPU is needed because Groq's servers will do the heavy lifting.



## Install Libraries

In [ ]:
# Install the required libraries for the notebook
%pip install -q langchain langchain-groq python-dotenv faiss-cpu langchain_huggingface beautifulsoup4 langgraph

## Configure Groq API Key

To use the Groq API, you need a key.

- Go to https://console.groq.com/keys and sign up.

- Create a new API Key and copy it.

If you're working with API keys (like for Groq, OpenAI, or Hugging Face), it's best to avoid hardcoding them directly in your notebook. Instead, use Colab’s secrets manager:

- Click the 🔑 key icon on the left sidebar (labeled "Secrets") in Colab.

- Add your secret (GROQ_API_KEY) there.



In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
#Using google.colab secrets
api_key = os.getenv("GROQ_API_KEY")

if not api_key:
    print("🛑 Groq API Key not found. Please make sure to set it up.")
else:
    print("✅ Groq API Key configured.")

## Selecting a LLM

We will set up the LLaMA 3 8B model from Meta — an open-source LLM — running via Groq’s optimized hardware backend.

Groq currently serves only open-weight models, which means you can inspect their architecture and, in many cases, run them locally if you choose. Examples include:

- llama3-8b-8192 and llama3-70b-8192 (Meta)

- gemma-7b-it (Google)

- mixtral-8x7b (Mistral)

- deepseek-r1-distill-llama-70b (DeepSeek & Meta)

[Groq’s Model Explorer](https://console.groq.com/docs/models) lists each model’s:

- Context window (maximum input length, e.g., 8192 tokens)

- Max output tokens (the maximum number of tokens a model can generate per call)

- Model family and version

- Inference speed estimates

This helps you pick the right model depending on your task — for example, summarization of long documents may benefit from a larger context window.

In [1]:
# Initialize the Chat model with Groq
# We'll use Llama3 8B, a fast and competent model
from langchain_groq import ChatGroq


llm = ChatGroq(model_name="llama-3.1-8b-instant", groq_api_key=api_key)


print("Language Model initialized with Groq.")

/Users/federicosvendsen/Documents/UPM/DeepLearning4NLP/1-7/script/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'api_key' is not defined

## LLM Hyperparameters

Language models accept not just a prompt, but also hyperparameters that modify their behavior. One of the most important is the temperature.

- **Low temperature** (e.g., 0.0 - 0.2): Makes the model more deterministic and predictable. It will choose the most likely words, which is ideal for fact-based tasks, summarization, or formatting.

- **High temperature** (e.g., 0.8 - 1.2): Increases randomness. The model might choose less likely words, encouraging creativity, diversity of ideas, and "personality." This is great for brainstorming, creative writing, or chatbots with character.

In [ ]:
# --- Low-Temperature Model (Predictable) ---
# Perfect for tasks requiring consistency and precision.
llm_low_temp = ChatGroq(model_name="llama-3.1-8b-instant",temperature=0.1, groq_api_key=api_key)


# --- High-Temperature Model (Creative) ---
# Ideal for generating new ideas or varied writing styles.
llm_high_temp = ChatGroq(model_name="llama-3.1-8b-instant",temperature=1, groq_api_key=api_key)

# The same prompt for both models
prompt = "Write a marketing slogan for a new GenAI application to make poetry."

print("--- 🤖 Low-Temperature Model (Predictable & Focused) ---")
response_low = llm_low_temp.invoke(prompt)
print(f"Slogan: {response_low.content}")

print("\n" + "="*50 + "\n")

print("--- 🎨 High-Temperature Model (Creative & Surprising) ---")
response_high = llm_high_temp.invoke(prompt)
print(f"Slogan: {response_high.content}")

Other hyperpameters include:

- 🎲**top_p**(Nucleus Sampling), An alternative way to control randomness. Instead of selecting from all possible tokens, it samples from the top p% of most likely next tokens. Typical range: 0.8 – 0.95. Works well in combination with temperature.
- 🧱 **max_tokens**, Sets the maximum number of tokens (words/pieces) in the model's response. Useful to limit verbosity or enforce concise output. Prevents runaway generation in long-form completions.
- ⛔ **stop**, Defines a list of strings where generation should halt. Useful for enforcing boundaries in structured formats (e.g., JSON, multi-turn dialogue, code snippets).

# A First Call to Groq: The Simple Way

Before diving into LangChain's advanced features, let's see the most basic way to call the model. We can put all our instructions—the task, the context, the input, and the desired output format—into a single, long string.

In this example, we simulate a real-world E-commerce use case: analyzing a customer's product review using a prompt-based approach with a large language model (LLM).





In [ ]:
# The customer review we want to analyze

simple_prompt = """
You are an expert sentiment analyst for an E-commerce company.
Your task is to analyze the customer's product review and provide your analysis.
The output must be a JSON object with three keys: "sentiment" (string), "summary" (string), and "rating" (integer from 1 to 5).
Only respond with the JSON object and nothing else.

Customer Review: "The keyboard is fantastic, the keys are smooth and the RGB lighting is spectacular.
The only downside is that the cable is a bit short, but otherwise, a great buy!"

JSON Output:
"""

print("--- Sending Simple Prompt to LLM ---")
response = llm.invoke(simple_prompt)
# The `invoke` method sends the `simple_prompt` string to the language model (llm).
# The model processes the prompt, generates a response,
# and returns that response as a BaseMessage object (containing the text output in "content").


print("\n--- Raw LLM Response ---")
print(response.content)

This works!

One major advantage of this method is that it does not require retraining or _fine-tuning_ the language model. Thanks to _in-context learning_, the model adapts to your task on the fly based solely on the prompt you provide.
For example, if tomorrow you decide that the rating should be from 1 to 3 instead of 1 to 5, you simply update the prompt, and you're done — no model retraining or redeployment needed.


**Only with this code, you can perform a wide variety of prototypes that use the LLM as a universal AI Backend, you simply have to ask for the task you want to do in the prompt and collect the response in your prototype.**

However, this simplistic use of prompting has several drawbacks:

- Brittle: It's hard to separate the instruction template from the input data.

- Hard to Manage: If you change the logic, you have to edit the string, which is error-prone.

- No Guarantees: The model might return a valid JSON string, but it's not guaranteed. Any small deviation could break downstream software that expects perfect JSON.

Now, let's see how LangChain helps us solve these problems more cleanly and reliably.



# Structured Prompting with LangChain

_Prompt Engineering_ refers to the practice of carefully crafting input prompts to guide a language model toward producing useful, accurate, or reliable outputs. Instead of retraining or fine-tuning a model, prompt engineering lets us control behavior and output by manipulating context.

LangChain provides a modular and clean framework to implement structured prompts, which is especially useful for building robust applications.



## The System Role
In modern LLM interfaces (like ChatGPT or Groq), a prompt is not just the user’s message. It often consists of multiple parts, including:

- **A system message**, which sets the tone, role, and global behavior of the model.

- **The user message**, which provides the actual task or input.

- **(Optionally) Assistant** or previous messages for conversational memory.

The `system role` is traditionally used to establish rules, instructions, or a “code of conduct” that the model must follow throughout the entire conversation. It acts as an “invisible voice” defining the AI assistant’s overall behavior, tone, boundaries, and internal guidelines before the user interacts. For example, you might instruct the model: “You are a medical assistant and never give legal diagnoses.” This role is essential to control the model’s behavior across different tasks — whether as a polite assistant, a strict data validator, or a customer support agent. Defining the system role helps produce more predictable and consistent outputs, which is crucial when integrating LLMs into production-level applications.

Note: Some APIs use the term `developer role` instead of system. While very similar, developer role messages often come with higher priority instructions from the app creator. Check your API’s docs for specifics.

Let’s revisit the same problem: classifying a customer review. We'll now separate the prompt into system and user roles. Additionally, the input data (review text) will appear in the user prompt using a placeholder that will be instantiated when the chain is invoked. A "chain" (of LangChain) is a single object that, when called, automatically fills the prompt with the input data, sends it to the model, and returns the output.




In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 1. Create the prompt template with system and user roles
system_prompt_text = """
You are an expert sentiment analyst for an E-commerce company.
Your task is to analyze a customer's product review and return the analysis in a structured JSON format.
The output must be a JSON object with three keys: "sentiment" (string), "summary" (string), and "rating" (integer from 1 to 5).
Only respond with the JSON object and nothing else.
"""
user_prompt_text = "Customer Review: {review}"

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt_text),
    ("user", user_prompt_text)
])
#   ChatPromptTemplate.from_messages creates a ChatPromptTemplate by defining a sequence of messages
#   Note that {review} in the user_prompt_text is a placeholder used with Python’s str.format() method.
#   It’s designed to be dynamically replaced with the actual customer review text when generating the prompt.
#   This makes the prompt reusable for analyzing different reviews by simply inserting the specific review content into the placeholder.

# 2. Create our chain, which links the prompt to the model
chain = prompt | llm

#  What this means:
#  The prompt template defines how we format the input text (including placeholders for dynamic content).
#  The language model (llm) takes this formatted input and generates a response.
#  By combining them into a "chain," we create a single object that, when called,
#  automatically fills the prompt with the input data, sends it to the model, and returns the output.
#  This simplifies the workflow, making it easy to run the model with different inputs without rewriting code.

# 3. Let's test it!
customer_review = "The keyboard is fantastic, the keys are smooth and the RGB lighting is spectacular. The only downside is that the cable is a bit short, but otherwise, a great buy!"
response = chain.invoke({"review": customer_review})

# Here, we pass a dictionary that fills the placeholders in the prompt.
# The chain automatically constructs the final prompt by inserting the values
# into the template before sending it to the model.
# This is different from the earlier approach with llm.invoke(simple_prompt),
# where we manually crafted and sent the full prompt string without placeholders.

print("\n--- Raw LLM Response ---")
print(response.content)

Better... but there's still a lot of additional information beyond the requested JSON. This makes it difficult to connect with other software that's only expecting that format.

## Text Classification with Guaranteed JSON Output


Now, using LangChain, we’ll define the output format of the LLM.

The   easiest and most reliable way to get structured outputs is using the method `with_structured_output()`  implemented for models that provide native APIs for structuring outputs.

The method returns a model-like Runnable, except that instead of outputting strings or messages it outputs objects corresponding to the given schema. The schema can be specified as a TypedDict class, JSON Schema or a Pydantic class. If TypedDict or JSON Schema are used then a dictionary will be returned by the Runnable, and if a Pydantic class is used then a Pydantic object will be returned.

Here, we will define our own  `Pydantic` class. Pydantic is a library for defining and validating structured data.  



In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# 1. Define the desired output structure using Pydantic
class SentimentAnalysis(BaseModel):
    sentiment: str = Field(description="The sentiment of the review, can be 'Positive', 'Negative', or 'Neutral'.")
    summary: str = Field(description="A concise, one-sentence summary of the customer's opinion.")
    rating: int = Field(description="A score from 1 to 5 based on the expressed sentiment.")

#  This Pydantic model defines the expected structure of the output from the LLM.
#  When the LLM returns a response, it is parsed and validated into an instance of SentimentAnalysis.
#  This means the output is not just raw JSON text, but a strongly-typed Python object
#  that can be easily used in your application with guaranteed structure and type safety.

# 2. Create a "structured" LLM that will enforce the output format of our Pydantic model
structured_llm = llm.with_structured_output(SentimentAnalysis)
#    .with_structured_output() is a method that takes a schema as input which specifies the names, types, and descriptions of the desired output attributes.
#    The method returns a model-like Runnable, except that instead of outputting strings or messages it outputs objects corresponding to the given schema.
#    The schema can be specified as a TypedDict class, JSON Schema or a Pydantic class


# 3. Create the prompt template with system and user roles
system_prompt_text = """
You are an expert sentiment analyst for an E-commerce company.
Your task is to analyze a customer's product review and return the analysis in a structured JSON format.
"""
user_prompt_text = "Customer Review: {review}"

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt_text),
    ("user", user_prompt_text)
])
#   ChatPromptTemplate.from_messages creates a ChatPromptTemplate by defining a sequence of messages
#   Note that {review} in the user_prompt_text is a placeholder used with Python’s str.format() method.
#   It’s designed to be dynamically replaced with the actual customer review text when generating the prompt.
#   This makes the prompt reusable for analyzing different reviews by simply inserting the specific review content into the placeholder.

# 4. Create our chain, which links the prompt to the model
chain = prompt | structured_llm

#  What this means:
#  The prompt template defines how we format the input text (including placeholders for dynamic content).
#  The language model (structured_llm) takes this formatted input and generates a response.
#  By combining them into a "chain," we create a single object that, when called,
#  automatically fills the prompt with the input data, sends it to the model, and returns the output.
#  This simplifies the workflow, making it easy to run the model with different inputs without rewriting code.

# 5. Let's test it!
customer_review = "The keyboard is fantastic, the keys are smooth and the RGB lighting is spectacular. The only downside is that the cable is a bit short, but otherwise, a great buy!"
response = chain.invoke({"review": customer_review})

# Here, we pass a dictionary that fills the placeholders in the prompt.
# The chain automatically constructs the final prompt by inserting the values
# into the template before sending it to the model.
# This is different from the earlier approach with llm.invoke(simple_prompt),
# where we manually crafted and sent the full prompt string without placeholders.

print(f"Original Review: '{customer_review}'\n")
print("--- Structured Analysis (JSON) ---")
print(f"\nType of response object: {type(response)}")
print(f"Extracted rating: {response.rating}")
print(f"Printing the response object: {response}")
# Note: Since SentimentAnalysis inherits from Pydantic's BaseModel, it provides a `.json()` method
# that allows easy conversion of the object to a JSON-formatted string
print(f"Printing the response object after convert it to JSON: {response.json()}")

As you can see, the model has followed the instructions perfectly, returning a Pydantic object that can be easily used or converted to JSON. This is much more robust!


Software engineering largely focuses on organizing code in a way that improves maintainability and scalability as projects grow. LangChain embodies this principle by providing modular components that help build, manage, and scale complex AI applications more effectively.

# In-Context Learning: Zero-Shot, One-Shot, and Few-Shot
LLMs can perform tasks differently based on the examples we provide in the prompt. This is called "in-context learning."

Let's illustrate this with a simple task: extracting the name of a technology from a text.

This can be considered a type of word-level classification (also called token classification). This is similar to Named Entity Recognition (NER), where tokens are classified into entity types like Person, Location, Organization, etc. Here, the classes are simpler — just “technology” vs “non-technology.”

## Zero-Shot Prompting  
We give it no examples. We rely on the model's pre-existing knowledge to perform the task.

In [ ]:
# Task: Extract the name of the Python library mentioned.

prompt_zero_shot = ChatPromptTemplate.from_messages([
    ("system", "Extract the main technology or library mentioned in the following text."),
    ("user", "Text: {input}")
])

chain_zero_shot = prompt_zero_shot | llm

text = "For data processing, people often use Pandas."
response = chain_zero_shot.invoke({"input": text})

print("--- Zero-Shot ---")
print(f"Input Text: '{text}'")
print(f"Model Response: {response.content}")

That’s fine. However, I only want the technology name, not an explanation. I could specify this in the system prompt or use `.with_structured_output` as before, but let’s try teaching the model with examples instead.

## One-Shot Prompting

We provide a single example to show the model exactly what we want.

`HumanMessage` and `AIMessage`  are special message types used in LangChain (and similar frameworks) to clearly distinguish between the messages coming from the human user and those coming from the AI model within a conversation or prompt.


In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

# Define a prompt with one example (one-shot learning) to guide the model.
prompt_one_shot = ChatPromptTemplate.from_messages([
    # System message: gives context and instructs the model what to do.
    ("system", "Extract the main technology or library mentioned in the following text."),

    # Example interaction we provide to show the expected format of the output.
    HumanMessage(content="Text: 'I love visualizing data with Matplotlib.'"),
    AIMessage(content="Matplotlib"),

    # The new input text, with a placeholder to be filled dynamically.
    ("user", "Text: {input}")
])

# Create a chain that links the prompt template with the language model (LLM).
chain_one_shot = prompt_one_shot | llm

# New text input to analyze, which will fill the {input} placeholder in the prompt.
new_text = "We have deployed our service in a Kubernetes cluster."

# Invoke the chain by passing the input dictionary to fill the placeholder.
response = chain_one_shot.invoke({"input": new_text})

# Print the input and the model's response.
print("\n--- One-Shot ---")
print(f"Input Text: '{new_text}'")
print(f"Model Response: {response.content}")

Great! We want the output to be just the technology name, without any extra explanation. This concise output is ideal for directly displaying in the GUI of my AI prototype.

##  Few-Shot Prompting  
We provide several examples. This is very useful for complex tasks or when we want a very specific output format.

In [ ]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field


# Define a small set of example input-output pairs to guide the model
examples = [
    {"input": "I love visualizing data with Matplotlib.", "output": "Matplotlib"},
    {"input": "For machine learning, I use Scikit-learn.", "output": "Scikit-learn"},
    {"input": "Our backend is built with Django.", "output": "Django"},
]

# Create a prompt template for each example, mapping user input to AI output
example_prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),  # Placeholder for the example input text
    ("ai", "{output}"),   # Placeholder for the corresponding example output (technology)
])

# Use FewShotChatMessagePromptTemplate to automatically insert examples into the previous prompt template
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# Construct the full prompt that the model will receive:
# - system message sets the task instructions
# - few-shot examples are included to show the model how to respond
# - user message with the new input to analyze
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the main technology or library mentioned in the text. Only return the name."),
    few_shot_prompt,  # Insert the few-shot examples here for in-context learning
    ("user", "{input}"),  # Placeholder for the actual user input text
])

# 1. Define the desired output structure using Pydantic
class SentimentAnalysis(BaseModel):
    technology: str = Field(description="Name of the technology or library being mentioned in the text.")
    technology2: str = Field(description="Name of another technology or library tbeing mentioned in the text, if non pick one that could be mostly related to the first one.")

structured_llm = llm.with_structured_output(SentimentAnalysis)
#    .with_structured_output() is a method that takes a schema as input which specifies the names, types, and descriptions of the desired output attributes.
#    The method returns a model-like Runnable, except that instead of outputting strings or messages it outputs objects corresponding to the given schema.
#    The schema can be specified as a TypedDict class, JSON Schema or a Pydantic class


# Create a chain linking the prompt to the language model
chain_few_shot = final_prompt | structured_llm

# Example input text to test the few-shot prompt
new_text  = "We use pyspark to manage data."

# Invoke the chain, filling the input placeholder and getting the model's response
response = chain_few_shot.invoke({"input": new_text})

# Print results
print("\n--- Few-Shot ---")
print(f"Input Text: '{new_text}'")
print(f"Model Response: {response}")

In this example, we progressively construct a structured and reusable prompt by composing smaller building blocks — moving step-by-step from low-level data toward a high-level, abstract interface.

- Examples List. First, we define a list of example input-output pairs that show the model how to perform the task. This provides the data we want to use for in-context learning.

- Example Prompt Template. We then define how each example should be formatted using a simple ChatPromptTemplate. This defines the shape of each "mini-conversation" the model will see.

- Few-Shot Prompt Template. Using FewShotChatMessagePromptTemplate, we automatically apply the formatting from the example template to all our examples. This builds a reusable prompt component that inserts well-formatted examples into the final prompt.

- Final Prompt Template. We then build the full prompt by combining:

  * A system message that clearly defines the task ("Extract the main technology..."),

  * The few-shot examples we defined,

  *  A user message with a placeholder for new input.

This step-by-step, modular approach lets us design complex prompting logic in a clean and maintainable way, which is especially useful when integrating LLMs into real applications.  

...or you can always go back to the zero-shot Prompting code and simply paste your examples directly into the system message (worse maintenance).

##Chuck Norrys Jokes

Let's ask our LLM to provide [Chuck Norrys Jokes](https://psycatgames.com/es/magazine/conversation-starters/chuck-norris-jokes/) with zero-shot prompting.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Simple zero-shot prompt: we ask the model to generate a Chuck Norris joke in Spanish
prompt_zero_shot = ChatPromptTemplate.from_messages([
    ("system", "Eres un generador de chistes de Chuck Norris. Devuelve solo un chiste en español, sin explicaciones."),
    ("user", "Cuéntame un chiste de Chuck Norris.")
])

# Link prompt to model
chain_zero_shot = prompt_zero_shot | llm
# Call the chain with no extra variables (no placeholders have been defined to be replaced in the prompt)
response = chain_zero_shot.invoke({})

# Display the result
print("--- Zero-Shot with LLama 8B ---")
print("Input: Cuéntame un chiste de Chuck Norris.")
print(f"Model Response: {response.content}")

Then, using few-shot prompting, we'll try to force the model to use the formula "Chuck Norris walks into a bar" in the joke.


In [ ]:
# List of example Chuck Norris jokes in Spanish
examples = [
  {"input": "Cuéntame un chiste de Chuck Norris.", "output": "Chuck Norris entra en un bar. El hielo se derrite por respeto."},
  {"input": "Cuéntame un chiste de Chuck Norris.", "output": "Chuck Norris entra en un bar. El camarero le pregunta qué quiere. Chuck lo mira. El camarero se convierte en cerveza."},
  {"input": "Cuéntame un chiste de Chuck Norris.", "output": "Chuck Norris entra en un bar. Todos los clientes se convierten en abstemios por instinto de supervivencia."},
  {"input": "Cuéntame un chiste de Chuck Norris.", "output": "Chuck Norris entra en un bar. La barra se endereza sola."},
  {"input": "Cuéntame un chiste de Chuck Norris.", "output": "Chuck Norris entra en un bar. El WiFi mejora."},
  {"input": "Cuéntame un chiste de Chuck Norris.", "output": "Chuck Norris entra en un bar. Nadie lo mira a los ojos. Ni los espejos."},
  {"input": "Cuéntame un chiste de Chuck Norris.", "output": "Chuck Norris entra en un bar. El bar se convierte en biblioteca por respeto a su silencio."},
  {"input": "Cuéntame un chiste de Chuck Norris.", "output": "Chuck Norris entra en un bar. El happy hour termina. El bar se pone serio."},
  {"input": "Cuéntame un chiste de Chuck Norris.", "output": "Chuck Norris entra en un bar. Pide un vaso vacío. Se emborracha el vaso."},
  {"input": "Cuéntame un chiste de Chuck Norris.", "output": "Chuck Norris entra en un bar. El bar sale corriendo."},
  {"input": "Cuéntame un chiste de Chuck Norris.", "output": "Chuck Norris entra en un bar. El DJ pone silencio."},
  {"input": "Cuéntame un chiste de Chuck Norris.", "output": "Chuck Norris entra en un bar. Se sirve solo. El vaso le da propina."},
  {"input": "Cuéntame un chiste de Chuck Norris.", "output": "Chuck Norris entra en un bar. El suelo se convierte en alfombra roja."},
  {"input": "Cuéntame un chiste de Chuck Norris.", "output": "Chuck Norris entra en un bar. El bartender se disculpa por no haberlo hecho famoso antes."},
  {"input": "Cuéntame un chiste de Chuck Norris.", "output": "Chuck Norris entra en un bar. Las luces se apagan para no deslumbrarlo."},
  {"input": "Cuéntame un chiste de Chuck Norris.", "output": "Chuck Norris entra en un bar. El letrero cambia a 'Museo'."},
  {"input": "Cuéntame un chiste de Chuck Norris.", "output": "Chuck Norris entra en un bar. El menú se resume a 'lo que él quiera'."},
  {"input": "Cuéntame un chiste de Chuck Norris.", "output": "Chuck Norris entra en un bar. El reloj se detiene para no hacerle perder el tiempo."},
  {"input": "Cuéntame un chiste de Chuck Norris.", "output": "Chuck Norris entra en un bar. La cerveza se enfría por sí sola del susto."},
  {"input": "Cuéntame un chiste de Chuck Norris.", "output": "Chuck Norris entra en un bar. El bar se convierte en gimnasio por reflejo condicionado."}
]

# Format for each example in the prompt
example_prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),
    ("ai", "{output}")
])

# Combine examples using FewShotChatMessagePromptTemplate
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

# Build the final prompt with system instructions + examples + user input
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un generador de chistes de Chuck Norris. Devuelve solo un chiste en español, sin explicaciones."),
    few_shot_prompt,  # Insert example dialogues
    ("user", "{input}")  # Placeholder for the actual user request
])

# Link the final prompt to the model
chain_few_shot = final_prompt | llm

# Input for a new joke
text = "Cuéntame un chiste de Chuck Norris."
response = chain_few_shot.invoke({"input": text})

# Display the result
print("\n--- Few-Shot ---")
print(f"Input: {text}")
print(f"Model Response: {response.content}")

#Conclusions and Next Steps

In this notebook, we gradually evolved from a simple prompt. Along the way, we learned how to:

- Using the Groq API to choose, configure, load, and invoke an LLM programmatically—enabling generative AI–powered software development.

- Craft more reliable and informative prompts using LangChain’s ChatPromptTemplate.

- Apply zero-shot, one-shot, and few-shot examples for better control over model behavior.



**Next Steps**

- Try to summarize a long text with LangChain.

- Manage conversation history with structured memory using LangGraph.

- Retrieval-Augmented Generation (RAG): Enhance your chatbot by connecting it to external knowledge sources (e.g., documents, databases). This lets it retrieve relevant context before generating a response—ideal for question answering, support bots, and knowledge assistants.

